# Сравнение свёрточного и полносвязного слоев

Давайте оценим количество ресурсов, которое требуется для обработки одного изображения из CIFAR-10 при помощи сверточного и полносвязного слоя.

Пусть сверточный слой будет содержать $6$ фильтров размером $3 \times 3 $, `padding = 1`, `stride = 1`, а полносвязный — $6$ выходов (как если бы мы делали классификацию $6$-ти классов).

<img src ="https://ml.gan4x4.ru/msu/dev-2.2/L06/out/conv_vs_linear.png" width="900">

## Сколько обучаемых праметров (весов) у свёрточного слоя?

Количество параметров в одном фильтре: $C_{in}\times K_{h}\times K_{w} +1 = 3 \times 3 \times 3 + 1 = 28$

Количество фильтров $C_{out} = 6$

Итого: $(C_{in}\times K_{h}\times K_{w} +1) \times C_{out} = 28 \times 6 = 168$



In [1]:
import torch
from torch.nn import Conv2d


def get_params_count(module):
    weights_count = 0
    # Get all model weights: kernels + biases
    for p in module.parameters():
        print(p.shape)
        # torch.prod - multiply all values in tensor
        weights_count += torch.tensor(p.shape).prod()
    print("Total weights", weights_count.item())


conv = Conv2d(3, 6, 3, bias=True)
get_params_count(conv)

torch.Size([6, 3, 3, 3])
torch.Size([6])
Total weights 168


## Сколько обучаемых праметров у полносвязного слоя?

1. Данные вытягиваем в вектор:

$\text{inputs_count} = C_{in} \times H_{in} \times W_{in}  = 3*32*32 = 3072$

2. Каждый нейрон (их $6$ шт.) выходного слоя хранит вес для каждого элемента входа ($3072$) и еще одно смещение:

$(\text{inputs_count} + 1) \times \text{outputs_count} = (3072 + 1) \times 6 = 18\ 438$


In [2]:
from torch.nn import Linear

linear = Linear(3072, 6, bias=True)
get_params_count(linear)

torch.Size([6, 3072])
torch.Size([6])
Total weights 18438


То есть для хранения весов такого линейного слоя нужно в $\approx 100$ раз больше памяти.

## Сколько вычислительных ресурсов требуется полносвязному слою?

*Считаем только умножения, т. к. (умножение + сложение = 1 [FLOP 📚[wiki]](https://en.wikipedia.org/wiki/FLOPS)).*

В полносвязном слое каждый вход умножается на свой вес один раз и количество умножений совпадает с количеством весов за вычетом сложения со смещением:

 $C_{in} × H_{in} × W_{in} × \text{outputs_count}  = 3 × 32 \times 32 × 6 = 18 \ 432 $



## Сколько вычислительных ресурсов требуется свёрточному слою?

1. Разовое применение фильтра эквивалентно применению линейного слоя с таким же количеством весов:

$C_{in} \times K_{h} \times K_{w} \times C_{out} = 3 \times 3 \times 3 \times 6 = 162$

Т. е. умножаем каждый вес фильтра на вход.

2. Сдвигаем фильтр и повторяем п. 1 для каждой точки на карте признаков:

$C_{in} \times K_{h} \times K_{w} \times C_{out} \times H_{out} \times W_{out}   = 162 \times 30 \times 30  = 145\ 800 $

То есть количество операций в $\approx 10$ раз больше, чем у полносвязного слоя.

**Выводы**:
выигрыш по количеству параметров при использовании свёрточного слоя омрачается большим количеством операций перемножения. Это было проблемой в течение долгого времени, пока вычисление операции свёртки не перевели на видеокарты (Graphical Processing Unit). При выполнении свёртки одного сегмента не требуется информация о результатах свёртки на другом сегменте, поэтому данные операции можно выполнять параллельно, с чем как раз прекрасно справляются видеокарты.

В PyTorch можно управлять вычислительными ресурсами и выбирать устройства где производить вычисления.

Если графический процессор доступен (то есть PyTorch может использовать CUDA), то вычисления будут производиться на нём. В противном случае, будет использоваться CPU. Обычно используют такую конструкцию:


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Создадим свёрточную нейронную сеть с одним входным каналом и шестью выходными каналами, а затем модель разместим на выбранное устройство (GPU или CPU) с помощью метода `.to(device)`:

In [4]:
model = Conv2d(in_channels=1, out_channels=6, kernel_size=3)
model.to(device)  # send model to device

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

Данные должны быть размещены на том же устройстве, что и модель:

In [5]:
dummy_input = torch.randn(1, 1, 5, 5)
out = model(dummy_input.to(device))  # send data to GPU too!

 После вычислений на GPU, часто возникает необходимость передать результат обратно в основную память, чтобы его можно было использовать для дальнейшей обработки, например для визуализации или сохранения. Это делается с помощью метода `.cpu()`:


In [6]:
out = out.cpu()  # move data back to main memory